The NOAA HRRR is a real-time 3km resolution, hourly updated, cloud-resolving, convection-allowing atmospheric model, initialized by 3km grids with 3km radar assimilation.

This notebook provides an example of accessing HRRR data, including (1) finding the data file corresponding to a date and time, (2) retrieving a portion of that file from blob storage which includes the surface temperature variable, (3) opening the file using the xarray library, and (4) rendering an image of the forecast.

This dataset is documented at http://aka.ms/ai4edata-hrrr.

In [1]:
import climatedata_functions as climf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
metadata = pd.read_csv("../data/metadata.csv")

In [ ]:
ds = climf.get_ds()#only words with recent dates

### Make metadafile with gridpoints

In [ ]:
#tranform coordinates for longitude
metadata['longitude_trans'] = metadata['longitude']+360 
#add columns for gridpoints
metadata['x_grid'] = ''
metadata['y_grid'] = ''

#make new metadatafile and save as cvs 
#metadata_new = climf.save_grids(metadata.head(), ds) #(takes approx 6 minutes)
#metadata_new.to_csv('../data/metadata_grids.csv')


In [ ]:
#load new metadata file that includes the gridpoints
metadata = pd.read_csv("../data/metadata_grids.csv")

In [ ]:
metadata.head(10)

### Get temperatures for gridpoints

Make function that gets the mean/median temperature for one date for all the places sampled at that date (to save time) and the previos x days

In [5]:
metadata = pd.read_csv('../data/metadata_temp_12_complete_and_temp1start.csv')


In [7]:
from datetime import timedelta
import warnings
warnings.simplefilter(action='ignore')

start_date = climf.get_start_date(metadata.date[0])
date_index_list = metadata.index[metadata.date == start_date]
#print(date_index_list)
temp_list = []
done_list = []
days = 7
hour = 6 #which our of the day

metadata['temp_06'] = ''
for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        #print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date, hour)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp_06.loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

2018-05-14
2016-08-31
2020-11-19
2016-08-24
2019-07-23
2021-08-23
2017-11-15
2020-06-10
2014-08-12
2018-06-27
2013-11-06
2015-08-24
2014-11-01
2021-10-18
2015-08-26
2019-08-26
2018-01-08
2015-08-18
2017-07-28
2015-07-07
2018-06-13
2018-01-09
2013-07-09
2020-08-13
2013-08-17
2018-08-13
2017-03-09
2013-07-29
2017-12-05
2019-06-06
2020-02-26
2018-04-11
2020-05-18
2016-07-20
2018-08-27
2020-03-18
2019-01-17
2016-06-21
2015-01-13
2016-06-08
2016-03-08
2013-01-15
2021-07-14
2017-06-19
2016-04-13
2019-07-02
2019-05-22
2014-02-28
2014-07-15
2017-08-01
2019-09-24
2017-09-26
2016-01-19
2013-08-20
2017-08-02
2016-06-22
2015-06-10
2018-04-03
2016-04-06
2016-05-17
2017-10-05
2013-07-16
2016-07-19
2014-12-03
2016-08-05
2020-05-12
2015-09-25
2017-06-12
2017-03-31
2019-03-19
2017-08-28
2015-04-22
2019-08-22
2016-05-24
2016-12-13
2015-04-14
2020-06-24
2014-07-22
2020-08-11
2018-06-20
2015-11-16
2017-08-25
2020-11-10
2020-07-15
2013-09-23
2013-02-12
2015-06-22
2015-05-06
2014-06-23
2021-08-16
2016-09-20

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [15]:
done_list.pop()#remove last element (in case it didn work with thst)
len(done_list)

1216

In [26]:
#done_list_save = pd.Series(done_list)
#done_list_save.to_csv('../data/done_list_temp.csv')

In [16]:
import warnings
warnings.simplefilter(action='ignore')

start_date = climf.get_start_date(metadata.date[0])
date_index_list = metadata.index[metadata.date == start_date]
#print(date_index_list)
temp_list = []
#done_list = [] deactivate because we want to continue from the old donelist
days = 7
hour = 12 #which our of the day

for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        #print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date, hour)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp_01.loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

2021-06-03
2014-06-18
2016-05-12
2016-04-28
2017-11-06
2017-07-16
2014-02-04
2019-12-12
2017-03-30
2015-05-27
2021-01-21
2016-07-26
2018-10-22
2020-08-10
2014-06-09
2015-08-04
2019-03-13
2017-10-24
2014-09-24
2020-06-30
2014-04-26
2018-04-16
2018-04-19
2015-02-23
2018-07-26
2013-03-19
2020-09-25
2021-07-01
2017-05-10
2020-03-05
2014-02-24
2017-06-23
2013-07-08
2019-06-29
2017-03-06
2017-04-24
2021-05-12
2021-04-06
2013-01-26
2016-11-16
2020-10-27
2014-10-21
2018-10-24
2017-04-06
2018-05-02
2015-08-08
2020-03-19
2020-08-28
2013-04-22
2016-03-21
2021-04-12
2013-08-10
2015-06-17
2016-09-03
2020-09-18
2020-10-26
2016-08-13
2019-07-31
2017-05-22
2018-08-29
2020-02-10
2013-05-20
2020-05-05
2016-02-19
2015-03-01
2014-02-13
2016-04-26
2015-10-09
2014-10-01
2017-01-31
2019-12-09
2013-05-13
2014-02-07
2021-01-20
2014-08-10
2014-06-03
2013-06-05
2017-07-14
2018-09-21
2021-12-29
2015-01-28
2019-06-20
2016-07-15
2018-12-12
2021-08-04
2014-09-02
2018-05-24
2018-05-16
2013-03-12
2014-06-05
2021-12-16

In [22]:
print(metadata.date.nunique())
metadata.columns

1637


Index(['Unnamed: 0', 'Unnamed: 0.1', 'uid', 'latitude', 'longitude', 'date',
       'split', 'longitude_trans', 'x_grid', 'y_grid', 'temp', 'temp_01',
       'temp_12'],
      dtype='object')

In [23]:
from datetime import timedelta
import warnings
warnings.simplefilter(action='ignore')

start_date = climf.get_start_date(metadata.date[0])
date_index_list = metadata.index[metadata.date == start_date]
#print(date_index_list)
temp_list = []
done_list = []
days = 7
hour = 6 #which our of the day

metadata['temp_06'] = ''
for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        #print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date, hour)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp_06.loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

2018-05-14
2016-08-31
2020-11-19
2016-08-24
2019-07-23
2021-08-23
2017-11-15
2020-06-10
2014-08-12
2018-06-27
2013-11-06
2015-08-24
2014-11-01
2021-10-18
2015-08-26
2019-08-26
2018-01-08
2015-08-18
2017-07-28
2015-07-07
2018-06-13
2018-01-09
2013-07-09
2020-08-13
2013-08-17
2018-08-13
2017-03-09
2013-07-29
2017-12-05
2019-06-06
2020-02-26
2018-04-11
2020-05-18
2016-07-20
2018-08-27
2020-03-18
2019-01-17
2016-06-21
2015-01-13
2016-06-08
2016-03-08
2013-01-15
2021-07-14
2017-06-19
2016-04-13
2019-07-02
2019-05-22
2014-02-28
2014-07-15
2017-08-01
2019-09-24
2017-09-26
2016-01-19
2013-08-20
2017-08-02
2016-06-22
2015-06-10
2018-04-03
2016-04-06
2016-05-17
2017-10-05
2013-07-16
2016-07-19
2014-12-03
2016-08-05
2020-05-12
2015-09-25
2017-06-12
2017-03-31
2019-03-19
2017-08-28
2015-04-22
2019-08-22
2016-05-24
2016-12-13
2015-04-14
2020-06-24
2014-07-22
2020-08-11
2018-06-20
2015-11-16
2017-08-25
2020-11-10
2020-07-15
2013-09-23
2013-02-12
2015-06-22
2015-05-06
2014-06-23
2021-08-16
2016-09-20

In [53]:
#metadata.to_csv('../data/metadata_temp_12_01_06_complete.csv')